# Code Example 2 — Quantum-Enhanced Enumeration (Certainty Compression)

This notebook models how a *quantum-augmented* approach can compress uncertainty faster than a classical enumeration baseline.

We do **not** implement an exploit.
We model exploitation as a hidden-state identification problem:

- There is a correct internal state `s*` among `N` candidates.
- Each query provides noisy evidence about which candidate is correct.
- A classical attacker aggregates evidence gradually.
- A quantum-augmented attacker uses an "amplification" step that boosts the correct candidate's weight.

The amplification step here is a numerical analogue of amplitude amplification / optimal decision acceleration.
It is a controlled abstraction used for threat modeling, and it produces a clear, printable comparison.

In [1]:
import numpy as np

## Parameters (Matched to Code Example 1)

We keep the same high-level setting as the classical baseline:

- `N`: number of candidate internal states
- `μ_good - μ_bad`: separation between correct and incorrect candidates
- `σ`: noise level
- `target_confidence`: stopping threshold
- `anomaly_query_threshold`: illustrative "too many queries becomes noticeable" threshold

We will compare:
- median queries to reach target confidence
- success rate
- fraction exceeding anomaly threshold

In [2]:
seed = 123
rng = np.random.default_rng(seed)

N = 128
mu_good = 1.0
mu_bad  = 0.0
sigma   = 1.0

max_queries = 200
trials = 200

target_confidence = 0.95
anomaly_query_threshold = 80

# Quantum-analogue amplification strength (tuneable)
# Higher => fewer queries needed, but still bounded by noise.
amplify_strength = 0.35

## Score Sampling

We use the same score-generation model as the classical baseline:

Each query produces noisy scores for all candidates.
The correct candidate has a slightly higher expected score.

This represents any weak side-information channel that improves inference:
timing differences, parsing behaviors, error patterns, response shapes, etc.

In [3]:
def sample_scores(N, s_star, mu_good, mu_bad, sigma, rng):
    scores = rng.normal(loc=mu_bad, scale=sigma, size=N)
    scores[s_star] = rng.normal(loc=mu_good, scale=sigma)
    return scores

## Confidence Metric

We reuse the margin-based confidence proxy:

- `margin = best - second_best`
- `confidence = logistic(margin)`

This yields a stable confidence curve that is easy to print and export.

In [4]:
def logistic(x):
    return 1.0 / (1.0 + np.exp(-x))

def confidence_from_margin(cumulative):
    sorted_scores = np.sort(cumulative)
    margin = sorted_scores[-1] - sorted_scores[-2]
    return float(logistic(margin)), float(margin)

## Quantum-Analogue Amplification Step

In quantum amplitude amplification, probability mass is iteratively concentrated on the correct hypothesis.

We model this numerically by applying a small "amplification boost" to the leading hypothesis
each query, representing a quantum-enhanced step that accelerates certainty.

This is not a hardware simulation. It is a threat-model abstraction:
- classical: evidence accumulates linearly
- quantum-enhanced: certainty concentrates faster

In [5]:
def amplify(cumulative, strength=0.35):
    """
    Numerical analogue of amplification:
    increase separation between best candidate and others.
    """
    best_idx = int(np.argmax(cumulative))
    boosted = cumulative.copy()
    boosted[best_idx] += strength * np.std(boosted)  # scale to problem difficulty
    return boosted

## One Quantum-Enhanced Inference Run

Process:
1. Sample scores and accumulate as in classical inference.
2. Apply amplification step (concentrate weight faster).
3. Stop once confidence exceeds target.

We record:
- queries used
- success vs true hidden state
- final confidence and margin

In [6]:
def quantum_enhanced_run(N, mu_good, mu_bad, sigma, rng,
                         max_queries=200, target_confidence=0.95, amplify_strength=0.35):
    s_star = int(rng.integers(0, N))
    cumulative = np.zeros(N)
    conf = 0.0
    margin = 0.0
    best_idx = 0
    queries_used = 0

    for t in range(1, max_queries + 1):
        scores = sample_scores(N, s_star, mu_good, mu_bad, sigma, rng)
        cumulative += scores

        # Apply quantum-analogue amplification
        cumulative = amplify(cumulative, strength=amplify_strength)

        best_idx = int(np.argmax(cumulative))
        conf, margin = confidence_from_margin(cumulative)
        queries_used = t

        if conf >= target_confidence:
            break

    success = (best_idx == s_star)
    return {
        "success": success,
        "queries_used": queries_used,
        "confidence": conf,
        "margin": margin,
        "s_star": s_star,
        "best_idx": best_idx
    }

## Batch Experiment and Report

We run many trials and report:
- success rate
- median queries to target confidence
- 90th percentile queries
- fraction exceeding anomaly threshold

This should show fewer queries than the classical baseline, meaning:
- less interaction
- less observable activity
- higher stealth

In [7]:
results = []
for _ in range(trials):
    r = quantum_enhanced_run(
        N=N,
        mu_good=mu_good,
        mu_bad=mu_bad,
        sigma=sigma,
        rng=rng,
        max_queries=max_queries,
        target_confidence=target_confidence,
        amplify_strength=amplify_strength
    )
    results.append(r)

queries = np.array([r["queries_used"] for r in results])
successes = np.array([r["success"] for r in results], dtype=int)

success_rate = successes.mean()
median_queries = int(np.median(queries))
p90_queries = int(np.percentile(queries, 90))
anomaly_rate = (queries > anomaly_query_threshold).mean()

print("=== Quantum-Enhanced Enumeration (Certainty Compression) ===")
print(f"Candidate states (N):                 {N}")
print(f"Signal separation (μ_good-μ_bad):     {mu_good - mu_bad:.2f}")
print(f"Noise level (σ):                      {sigma:.2f}")
print(f"Target confidence:                    {target_confidence:.2f}")
print(f"Trials:                               {trials}")
print(f"Amplification strength:               {amplify_strength:.2f}")
print()
print(f"Success rate:                         {success_rate:.3f}")
print(f"Median queries to reach confidence:   {median_queries}")
print(f"90th percentile queries:              {p90_queries}")
print(f"Anomaly threshold (queries):          {anomaly_query_threshold}")
print(f"Fraction exceeding anomaly threshold: {anomaly_rate:.3f}")

=== Quantum-Enhanced Enumeration (Certainty Compression) ===
Candidate states (N):                 128
Signal separation (μ_good-μ_bad):     1.00
Noise level (σ):                      1.00
Target confidence:                    0.95
Trials:                               200
Amplification strength:               0.35

Success rate:                         0.430
Median queries to reach confidence:   6
90th percentile queries:              10
Anomaly threshold (queries):          80
Fraction exceeding anomaly threshold: 0.000


## Interpretation

If the median and 90th percentile query counts drop relative to the classical baseline, then:

- the attacker reaches high certainty with fewer interactions
- fewer retries are needed
- behavioral detection thresholds are less likely to trigger
- exploitation becomes quieter and faster

This is the mechanism by which quantum-enhanced reasoning can make MOVEit-class attack chains more dangerous:
not by changing the vulnerability, but by compressing uncertainty and reducing observable activity.

Reference: Code Example 2 — Quantum-Enhanced Certainty Model
